<a href="https://colab.research.google.com/github/viniciusrpb/cic0193_machinelearning/blob/main/cap5_2_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
import pandas as pd
import numpy as np

In [155]:
rawData = pd.read_csv('/content/sample_data/spam_dataset.csv')

In [156]:
rawData

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,word_freq_hpl,word_freq_george,word_freq_650,word_freq_lab,word_freq_labs,word_freq_telnet,word_freq_857,word_freq_data,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,target_class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.64,0.00,0.00,0.00,0.32,0.00,1.29,1.93,0.00,0.96,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.000,0.0,778.000,0.00,0.00,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.00,1.59,0.0,0.43,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,132.000,0.0,0.372,0.18,48.00,5114.000,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.00,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.12,0.00,0.06,0.06,0.0,0.0,0.01,0.143,0.0,276.000,184.00,0.01,9821.000,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.137,0.0,0.137,0.00,0.00,3537.000,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,135.000,0.0,135.000,0.00,0.00,3537.000,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,0.00,1.88,0.00,0.00,0.00,0.00,0.00,0.00,0.62,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.31,0.31,0.31,0.0,0.0,0.00,0.232,0.0,0.000,0.00,0.00,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.00,0.00,2.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,2.00,0.0,0.0,0.00,0.000,0.0,353.000,0.00,0.00,1555.000,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.80,0.30,0.00,0.00,0.00,0.00,0.90,1.50,0.00,0.30,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,1.20,0.0,0.0,102.00,718.000,0.0,0.000,0.00,0.00,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.00,1.93,0.00,0.32,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.32,0.00,0.32,0.0,0.0,0.00,57.000,0.0,0.000,0.00,0.00,1147.000,5,78,0


In [157]:
def gaussian(xi,media,std):

  denominador = (1.0 / (np.sqrt(2*np.pi)*std))
  numerador = np.exp( - ((xi-media)**2) / (2*std**2) )

  return numerador * denominador

In [158]:
def calculaMediaStd(X,n,d):

  medias = X.mean(axis=0)
  desvios_padroes = X.std(axis=0)

  return np.array([medias,desvios_padroes])

In [159]:
def prob_aposteriori_classes(stat_classes,qtde_classes,ap_classe,x_test,d):

  aposteriori = []

  for i in range(0,qtde_classes):

    ap = ap_classe[i]

    media = stat_classes[i][0]
    std = stat_classes[i][1]

    for atributo in range(0,d):
      ap = ap*gaussian(x_test[atributo],media[atributo],std[atributo])

    aposteriori.append(ap)
  
  return aposteriori

In [160]:
N,M = rawData.shape

# separa algumas para treinamento e outras para teste

spam = rawData.iloc[0:4,:]
notspam = rawData.iloc[4597:4601,:]

teste = pd.concat([spam,notspam])

treinamento = rawData.iloc[4:4597,:]

# da quantidade de total de atributos, desconta o atributo de classe (target_class)
M = M-1

In [161]:
labels = treinamento['target_class'].unique()

labels.sort()

print(labels)

[0 1]


In [162]:
X = treinamento.iloc[:,:-1].values
y = treinamento.iloc[:,-1].values

Treinamento: aprendizado das características/padrões dos dados que são armazenadas nos parâmetros do modelo (distribuição Gaussiana, média e desvio padrão)

In [163]:
apriori_classe = []
estatisticas_classe = []

id_classe = 0

for classe in labels:

  bool_classe = treinamento['target_class'] == classe

  int_classe_df = treinamento[bool_classe].iloc[:,:-1]

  X_classe = int_classe_df.values

  apriori_classe.append(len(X_classe)/N)

  estatisticas_classe.append(calculaMediaStd(X_classe,len(X_classe),M))

Predição utilizando o classificador

In [164]:
X_test = teste.iloc[:,:-1].values
y_test = teste.iloc[:,-1].values

In [165]:
for x in X_test:
  prob_aposteriori = prob_aposteriori_classes(estatisticas_classe,len(labels),apriori_classe,x,M)

  if prob_aposteriori[0] < prob_aposteriori[1]:
    print("Eh spam")
  else:
    print("Nao eh spam")

Eh spam
Eh spam
Eh spam
Eh spam
Nao eh spam
Nao eh spam
Eh spam
Eh spam


In [166]:
prob_aposteriori

[1.9658118998515695e-30, 1.0763421539117863e-26]

In [167]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

clf.fit(X,y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [168]:
print(clf.predict(X_test))

[1 1 1 1 0 1 1 0]


In [169]:
print(y_test)

[1 1 1 1 0 0 0 0]
